In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news2-0/news_2.0.csv
/kaggle/input/till-2000/summarized_news.csv
/kaggle/input/news-till-1000/summarized_news.csv


### Importing Libraries

In [2]:
from transformers import pipeline

import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

2024-04-21 17:52:29.468474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 17:52:29.468597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 17:52:29.634720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Importing Dataset

In [3]:
df = pd.read_csv('/kaggle/input/news2-0/news_2.0.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50000 non-null  int64 
 1   title       50000 non-null  object
 2   article     50000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


### Importing Model for Summarization(BART)

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_ckpt = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipeline('summarization', model=model_ckpt, device = device)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

### 10000 Samples

In [6]:
temp_df = df.iloc[20000:30000]
temp_df.reset_index(drop=True, inplace=True)

### Summarization Function

In [8]:
import torch
from transformers import pipeline

# Example function to generate segments of text within the maximum sequence length
def generate_segments(input_text):
    words = input_text.split()
    for i in range(0, len(words), 300):  # Adjust the segment length here
        yield ' '.join(words[i:i+300])   # Adjust the segment length here

def summarization(input_text):
    global count
    if(len(tokenizer.tokenize(input_text))<=1020):
        count += 1
        print(count, end=" ")
        return input_text

    output_summary = ""
    for segment in generate_segments(input_text):
        try:
            max_length = len(segment.split()) # Adjust max_length dynamically
            summarized_segment = pipe(segment, max_length=max_length, min_length=1, do_sample=False)[0]['summary_text']
            output_summary += summarized_segment
            
        except IndexError:
            pass
        except Exception as e:
            print(f"Error occurred: {e}")
    if len(output_summary.split()) >= 1020:
            output_summary = summarization(output_summary)
    count += 1
    print(count, end=" ")
    return output_summary
count = 0

In [ ]:
final_list = []
final_list.append(temp_df['article'].apply(summarization))

In [12]:
temp_df['summary'] = final_list[0]

### Summarization Result

In [13]:
temp_df

,title,article,summary
0,You Need to Care About Facebook Censoring an I...,Nick Ut's Pulitzer Prize-winning photograph of...,Nick Ut's Pulitzer Prize-winning photograph of...
1,Explainer: Counting the cost of the U.S.-China...,HONG KONG (Reuters) - U.S. President Donald Tr...,U.S. President Donald Trump's fresh threats of...
2,'Breaking Bad' Star Lavell Crawford Dropped 12...,Comedian and actor Lavell Crawford's not quite...,Comedian and actor Lavell Crawford's not quite...
3,"Daily Horoscope: November 09, 2016",Mars enters Aquarius today at 12:52 AM! Mars i...,Mars enters Aquarius today at 12:52 AM! Mars i...
4,"UK's May tries to calm Brexit rebels, says dea...",LONDON (Reuters) - Prime Minister Theresa May ...,LONDON (Reuters) - Prime Minister Theresa May ...
...,...,...,...
9994,Correspondent Takeover: Let's talk about Penns...,"Hey, VICE News fans, it’s me, Evan McMorris-Sa...","Hey, VICE News fans, it’s me, Evan McMorris-Sa..."
9995,The best TV episodes of 2015: from Walking Dea...,Even in our era of streaming television and bi...,There's still something wonderful about a sing...
9996,Toxic algae is killing thousands of fish in Fl...,Captain Billy Norris recently had to institute...,"Since early July, thousands of dead fish have ..."
9997,Saying Donald Trump is totally unlike mainstre...,"Donald Trump is weird. So weird, in fact, that...",Julian Zelizer: Donald Trump is a deeply unusu...


### Saving Dataset

In [14]:
temp_df.to_csv("till_30000.csv")